# Replica Set

A [ReplicaSet](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/)'s purpose is to maintain a stable set of replica Pods running at any given time.

ReplicaSet的目的是讓Pods副本在任何時間都能維持同一狀態

> 在minikube的環境下，access Cluster IP或是 access Pod IP的時候要用 minikube ssh, nodeIP 着不需要 minikubessh

In [210]:
from IPython.display import Image
Image(url="./img/replicaset.jpg", width=300, height=300)

## 什麼時候用ReplicaSet

A ReplicaSet ensures that a sepecified nubmer of pod replicas are running at any given time, it is suitable for Stateless application.

ReplicaSet可以保持指定數量的Pods副本在任何時間繼續運行.

## 什麼是無狀態應用 Stateless

Stateless 是指這個應用服務或功能，不因任何狀態而改變回傳的資料。
- eg: 沒有資料庫的static web

## 什麼是有狀態應用  StateFul

Stateful 則與 Stateless 相反，會因狀態而改變回傳的資料，像是Request的次數。
- eg: 有資料庫的 wordpress

## Replica Set

而ReplicaSet 正適合與建立stateless服務，因爲Pod在消滅後再建立的話，是不保存之前的狀態的。

ReplicaSet:
- 管理一群Pods
- 透過 Selector 選擇那些Pod 需要進行管理
- 每個 ReplicaSet 可以根據需求建立或刪除Pod使得副本達到期望值
- 利用template建立Container


In [211]:
# 確保你的minikube有啟動
! [[ -z $(kubectl get nodes) ]] && minikube start 
# 列印目前k8s節點
! kubectl get nodes                    

NAME       STATUS   ROLES           AGE    VERSION
minikube   Ready    control-plane   153m   v1.25.3


In [212]:
# Get the documentation of the resource and its fields
! kubectl explain replicaset

KIND:     ReplicaSet
VERSION:  apps/v1

DESCRIPTION:
     ReplicaSet ensures that a specified number of pod replicas are running at
     any given time.

FIELDS:
   apiVersion	<string>
     APIVersion defines the versioned schema of this representation of an
     object. Servers should convert recognized schemas to the latest internal
     value, and may reject unrecognized values. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#resources

   kind	<string>
     Kind is a string value representing the REST resource this object
     represents. Servers may infer this from the endpoint the client submits
     requests to. Cannot be updated. In CamelCase. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#types-kinds

   metadata	<Object>
     If the Labels of a ReplicaSet are empty, they are defaulted to be the same
     as the Pod(s) that the ReplicaSet manages. Standard object's metadata

In [213]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.replicas

KIND:     ReplicaSet
VERSION:  apps/v1

FIELD:    replicas <integer>

DESCRIPTION:
     Replicas is the number of desired replicas. This is a pointer to
     distinguish between explicit zero and unspecified. Defaults to 1. More
     info:
     https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller/#what-is-a-replicationcontroller


In [214]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.selector

KIND:     ReplicaSet
VERSION:  apps/v1

RESOURCE: selector <Object>

DESCRIPTION:
     Selector is a label query over pods that should match the replica count.
     Label keys and values that must match in order to be controlled by this
     replica set. It must match the pod template's labels. More info:
     https://kubernetes.io/docs/concepts/overview/working-with-objects/labels/#label-selectors

     A label selector is a label query over a set of resources. The result of
     matchLabels and matchExpressions are ANDed. An empty label selector matches
     all objects. A null label selector matches no objects.

FIELDS:
   matchExpressions	<[]Object>
     matchExpressions is a list of label selector requirements. The requirements
     are ANDed.

   matchLabels	<map[string]string>
     matchLabels is a map of {key,value} pairs. A single {key,value} in the
     matchLabels map is equivalent to an element of matchExpressions, whose key
     field is "key", the operator is "In", and th

In [215]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.template

KIND:     ReplicaSet
VERSION:  apps/v1

RESOURCE: template <Object>

DESCRIPTION:
     Template is the object that describes the pod that will be created if
     insufficient replicas are detected. More info:
     https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller#pod-template

     PodTemplateSpec describes the data a pod should have when created from a
     template

FIELDS:
   metadata	<Object>
     Standard object's metadata. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#metadata

   spec	<Object>
     Specification of the desired behavior of the pod. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#spec-and-status



In [216]:
# Get the documentation of a specific field of a resource
! kubectl explain replicaset.spec.template.spec.containers

KIND:     ReplicaSet
VERSION:  apps/v1

RESOURCE: containers <[]Object>

DESCRIPTION:
     List of containers belonging to the pod. Containers cannot currently be
     added or removed. There must be at least one container in a Pod. Cannot be
     updated.

     A single application container that you want to run within a pod.

FIELDS:
   args	<[]string>
     Arguments to the entrypoint. The container image's CMD is used if this is
     not provided. Variable references $(VAR_NAME) are expanded using the
     container's environment. If a variable cannot be resolved, the reference in
     the input string will be unchanged. Double $$ are reduced to a single $,
     which allows for escaping the $(VAR_NAME) syntax: i.e. "$$(VAR_NAME)" will
     produce the string literal "$(VAR_NAME)". Escaped references will never be
     expanded, regardless of whether the variable exists or not. Cannot be
     updated. More info:
     https://kubernetes.io/docs/tasks/inject-data-application/define-co

In [217]:
# 所使用的範例replicaset.yaml
! cat replicaset.yaml             

apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: hostname
  labels:
    app: hostname
spec:
  # modify replicas according to your case
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
  template:
    metadata:
      labels:
        tier: frontend
    spec:
      containers:
      - name: hostname
        image: docker.io/brokenpen/hostname:v1

- yaml
  - apiVersion
  - kind
  - metadata
  - spec
     - replicas # 指定要維持多少分replica
     - selector # 選擇器，根據 Pod 的 Labels
     - template # container template

In [218]:
# 透過yaml 建立 replicaSet
! kubectl apply -f replicaset.yaml            # k apply -f replicaset.yaml

replicaset.apps/hostname created


In [219]:
# 列印目前 replicaset 
! kubectl get replicaset

NAME                  DESIRED   CURRENT   READY   AGE
hostname              0         0         0       1s
hostname-5477bcd76c   3         3         3       31m
hostname-7c99c74d96   0         0         0       37m


` kubectl get replicaset `
- `NAME` ReplicaSet 的名稱
- `DESIRED` 目標 Replica 數量
- `CURRENT` 目前 Replica 數量
- `READY` 當前 Ready的 Replica 數量
---


In [220]:
# 列印目前 rs(replicaset) 並且顯示 Labels
! kubectl get rs --show-labels

NAME                  DESIRED   CURRENT   READY   AGE   LABELS
hostname              0         0         0       1s    app=hostname
hostname-5477bcd76c   3         3         3       31m   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   0         0         0       37m   app=hostname,pod-template-hash=7c99c74d96


In [221]:
# 列印目前 labels 有標註 tier:frontend 的 Pods
! kubectl get pods --selector=tier=frontend --show-labels -o wide

NAME             READY   STATUS        RESTARTS   AGE   IP       NODE       NOMINATED NODE   READINESS GATES   LABELS
hostname-4df4s   0/1     Terminating   0          1s    <none>   minikube   <none>           <none>            tier=frontend
hostname-d9wkm   0/1     Terminating   0          1s    <none>   minikube   <none>           <none>            tier=frontend
hostname-x8c2k   0/1     Terminating   0          1s    <none>   minikube   <none>           <none>            tier=frontend


In [222]:
! kubectl get pods \
  --selector=tier=frontend \
  --no-headers \
  -o custom-columns=":status.podIP"

<none>
<none>
<none>


In [223]:
# Ctrl+Shift+L
# minikube ssh
# ! curl 172.17.0.3:8080

In [224]:
# 列印目前 Pods 和其Labels
! kubectl get pods --show-labels

NAME                        READY   STATUS        RESTARTS   AGE   LABELS
hostname                    1/1     Running       0          13m   app=nginx,env=dev
hostname-4df4s              0/1     Terminating   0          2s    tier=frontend
hostname-5477bcd76c-bsgqs   1/1     Running       0          31m   app=hostname,pod-template-hash=5477bcd76c
hostname-5477bcd76c-k225l   1/1     Running       0          31m   app=hostname,pod-template-hash=5477bcd76c
hostname-5477bcd76c-mcz9p   1/1     Running       0          31m   app=hostname,pod-template-hash=5477bcd76c
hostname-d9wkm              0/1     Terminating   0          2s    tier=frontend
hostname-x8c2k              0/1     Terminating   0          2s    tier=frontend


In [225]:
! kubectl get pods \
  --selector=tier=frontend \
  --no-headers \
  -o custom-columns=":metadata.name"

hostname-4df4s
hostname-d9wkm
hostname-x8c2k


In [ ]:
! kubectl delete pods --selector=tier=frontend
! echo;
! kubectl get pods --selector=tier=frontend
! echo;

pod "hostname-4df4s" deleted
pod "hostname-d9wkm" deleted
pod "hostname-x8c2k" deleted



In [ ]:
! kubectl get pods --selector=tier=frontend 

In [ ]:
! kubectl scale --help

In [ ]:
! kubectl scale --replicas=5 replicaset hostname

In [ ]:
! kubectl get pods --selector=tier=frontend 

In [ ]:
! kubectl scale --replicas=4 replicaset hostname
! kubectl get pods --selector=tier=frontend 

In [ ]:
! kubectl scale --replicas=2 replicaset hostname
! kubectl get pods --selector=tier=frontend 

In [ ]:
#

In [ ]:
# Autoscale
! kubectl autoscale rs hostname --max=10 --min=2 --cpu-percent=100

In [ ]:
# 可是 minikube 好像是捉不到targets 的 實際 cpu-percent 
# 所以在我們的workshop裡面，沒有用
! kubectl get HorizontalPodAutoscaler

In [ ]:
! kubectl get hpa

In [ ]:
! kubectl delete hpa hostname
! kubectl autoscale --max=10 --min=4 --cpu-percent=50 -f replicaset.yaml

In [ ]:
! kubectl get pods --selector=tier=frontend 

In [ ]:
# 刪除所有 replicaset 
! kubectl delete replicaset --all

In [ ]:
# 刪除所有 replicaset 
! kubectl delete hpa --all